In [1]:
%gui qt5

In [91]:
!pip install imageio imageio-ffmpeg

In [7]:
import napari
from skimage import data
from pathlib import Path
from typing import Dict, List, Union, Tuple
import numpy as np
import cv2 as cv
import dask.arr


In [8]:
rawdir = Path(r"C:\Data\Zoo_Jwasp_test")
vidpaths = list(rawdir.glob("j*.avi"))
vidpath = vidpaths[0]
vidpath

WindowsPath('C:/Data/Zoo_Jwasp_test/jwasp0.avi')

In [9]:
class Video:
    def __init__(self, cap: cv.VideoCapture):
        self.cap = cap
        
    @classmethod
    def from_file(cls, fname: str):
        return cls(cap=cv.VideoCapture(str(fname)))
    
    @property
    def n_frames(self) -> int:
        return int(self.cap.get(cv.CAP_PROP_FRAME_COUNT))
    
    def __len__(self) -> int:
        return self.n_frames
    
    @property
    def height(self) -> int:
         return int(self.cap.get(cv.CAP_PROP_FRAME_HEIGHT))
        
    @property
    def width(self) -> int:
         return int(self.cap.get(cv.CAP_PROP_FRAME_WIDTH))
        
    @property
    def n_channels(self) -> int:
        return 3
    
    def get_frame(self, idx: int) -> np.ndarray:
        self.cap.set(cv.CAP_PROP_POS_FRAMES, idx)
        valid, frame = self.cap.read()
        return frame
    
    def __getitem__(self, item: Union[int]) -> np.ndarray:
        if isinstance(item, int):
            return self.get_frame(item)
        else:
            raise TypeError()
        
        
    # Numpy-like interface
    @property
    def shape(self) -> Tuple[int, int, int, int]:
        shape = (self.n_frames, self.height, self.width, self.n_channels)
        return shape
    
    @property
    def dtype(self):
        return np.uint8
    
    @property
    def ndim(self):
        return 4
    
    @property
    def size(self):
        return np.product(self.shape)
    
    def min(self):
        return 0
    
    def max(self):
        return 255

vid = Video.from_file(vidpath)
vid[40:42]

TypeError: 

In [ ]:
%debug

> c:\users\nickdg\appdata\local\temp\ipykernel_12704\419413586.py(37)__getitem__()



ipdb>  l


     32 
     33     def __getitem__(self, item: Union[int]) -> np.ndarray:
     34         if isinstance(item, int):
     35             return self.get_frame(item)
     36         else:
---> 37             raise TypeError()
     38 
     39 
     40     # Numpy-like interface
     41     @property
     42     def shape(self) -> Tuple[int, int, int, int]:



ipdb>  item


slice(40, 42, None)


## Utils

In [41]:
def get_all_capture_values(cap: cv.VideoCapture) -> Dict[str, float]:
    props = [prop for prop in dir(cv) if prop.startswith('CAP_PROP_')]
    vals =  {prop: capture.get(getattr(cv, prop)) for prop in props if capture.get(getattr(cv, prop)) != 0}
    return vals

get_all_capture_values(capture)

{'CAP_PROP_BACKEND': 1900.0,
 'CAP_PROP_BITRATE': 19683.0,
 'CAP_PROP_CODEC_EXTRADATA_INDEX': 1.0,
 'CAP_PROP_CODEC_PIXEL_FORMAT': 808596553.0,
 'CAP_PROP_FOURCC': 877677894.0,
 'CAP_PROP_FPS': 120.0,
 'CAP_PROP_FRAME_COUNT': 6325.0,
 'CAP_PROP_FRAME_HEIGHT': 1080.0,
 'CAP_PROP_FRAME_WIDTH': 1440.0,
 'CAP_PROP_HW_DEVICE': -1.0,
 'CAP_PROP_ORIENTATION_AUTO': 1.0,
 'CAP_PROP_POS_AVI_RATIO': 0.008333333333333333,
 'CAP_PROP_POS_FRAMES': 2.0,
 'CAP_PROP_POS_MSEC': 8.333333333333334,
 'CAP_PROP_SAR_DEN': 1.0,
 'CAP_PROP_SAR_NUM': 1.0,
 'CAP_PROP_STREAM_OPEN_TIME_USEC': -9.223372036854776e+18}

In [64]:
capture.set(cv.CAP_PROP_POS_FRAMES, 2000)
get_all_capture_values(capture)

{'CAP_PROP_BACKEND': 1900.0,
 'CAP_PROP_BITRATE': 19683.0,
 'CAP_PROP_CODEC_EXTRADATA_INDEX': 1.0,
 'CAP_PROP_CODEC_PIXEL_FORMAT': 808596553.0,
 'CAP_PROP_FOURCC': 877677894.0,
 'CAP_PROP_FPS': 120.0,
 'CAP_PROP_FRAME_COUNT': 6325.0,
 'CAP_PROP_FRAME_HEIGHT': 1080.0,
 'CAP_PROP_FRAME_WIDTH': 1440.0,
 'CAP_PROP_HW_DEVICE': -1.0,
 'CAP_PROP_ORIENTATION_AUTO': 1.0,
 'CAP_PROP_POS_AVI_RATIO': 0.008333333333333333,
 'CAP_PROP_POS_FRAMES': 2000.0,
 'CAP_PROP_POS_MSEC': 16666.666666666668,
 'CAP_PROP_SAR_DEN': 1.0,
 'CAP_PROP_SAR_NUM': 1.0,
 'CAP_PROP_STREAM_OPEN_TIME_USEC': -9.223372036854776e+18}

In [65]:
a, frame = capture.read()
a

True

In [54]:
viewer = napari.view_image(frame)
viewer

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 539.5, 719.5), zoom=0.5268981481481482, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 1080.0, 1.0), (0.0, 1440.0, 1.0)), current_step=(540, 720), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'frame' at 0x19bbbcaea30>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ]), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vertices=False, selection_box_dra

In [59]:
layer = viewer.layers[0]
layer

<Image layer 'frame' at 0x19bbbcaea30>

In [66]:
layer.data = frame

In [51]:
import matplotlib

ModuleNotFoundError: No module named 'matplotlib'

In [50]:
b.shape

(1080, 1440, 3)